In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('day2').getOrCreate()

In [ ]:
titanicdata = spark.read.csv("/mnt/adls/DSG/ingestion/ID/SWAT/dummy/titanic.csv", inferSchema=True, header=True)

In [ ]:
display(titanicdata)

In [ ]:
titanicdata.printSchema()

In [ ]:
titanicdata.columns

In [ ]:
Data2 = titanicdata.select(['Survived',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare']) 

In [ ]:
Data_final = Data2.na.drop()

In [ ]:
Data2.count()

In [ ]:
Data_final.count()

In [ ]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,OneHotEncoder,StringIndexer)

In [ ]:
# # Stringindexer allows covert string to number

# A B C
# 0 1 2

# One hot 

# 1 0 0
# 0 1 0
# 0 0 1

In [ ]:
gender_index = StringIndexer(inputCol='Sex', outputCol='Gender_index')

In [ ]:
index_1 = gender_index.fit(Data_final)

In [ ]:
indexed_data = index_1.transform(Data_final)

In [ ]:
display(indexed_data)

In [ ]:
gender_encoder = OneHotEncoder(inputCol='Gender_index', outputCol='Gender_encoded')

In [ ]:
encoded = gender_encoder.transform(indexed_data)

In [ ]:
display(encoded)

In [ ]:
assembler = VectorAssembler(inputCols=['Age',  'SibSp',  'Parch',  'Fare','Gender_encoded'], outputCol='features')

In [ ]:
dataformodel = assembler.transform(encoded)

In [ ]:
display(dataformodel)

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
model = LogisticRegression(featuresCol='features', labelCol='Survived')

In [ ]:
train_df,test_df = dataformodel.randomSplit([0.7,0.3])

In [ ]:
titanicmodel =model.fit(train_df)

In [ ]:
test_results = titanicmodel.transform(test_df)

In [ ]:
display(test_results)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Survived')

In [ ]:
my_eval.evaluate(test_results)

In [ ]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy', predictionCol="prediction", labelCol="Survived")

In [ ]:
acc_eval.evaluate(test_results)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
y_true = test_results.select(['Survived']).collect()
y_pred = test_results.select(['prediction']).collect()
print(classification_report(y_true, y_pred))  

In [ ]:
confusion_matrix(y_true, y_pred)

In [ ]:
clustdata = spark.read.csv("/mnt/adls/DSG/ingestion/ID/SWAT/dummy/seeds_dataset.csv", inferSchema=True, header=True)


In [ ]:
display(clustdata)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [ ]:
assembler = VectorAssembler(inputCols= clustdata.columns, outputCol='features')

In [ ]:
final_data = assembler.transform(clustdata)

In [ ]:
display(final_data)

In [ ]:
from pyspark.ml.feature import MinMaxScaler

In [ ]:
scaler = MinMaxScaler(inputCol='features', outputCol='scaledfeatures')

In [ ]:
scaler_model = scaler.fit(final_data)

In [ ]:
final_data2 = scaler_model.transform(final_data)

In [ ]:
display(final_data2)

In [ ]:
kmeans = KMeans(featuresCol='scaledfeatures',k=3)

In [ ]:
kmeansmodel=kmeans.fit(final_data2)

In [ ]:
witcluster_tag =kmeansmodel.transform(final_data2)

In [ ]:
display(witcluster_tag)